In [6]:
import sys
sys.path.append("gaia-surrogate")
import torch
from gaia.models import TrainingModel
from gaia.training import get_checkpoint_file
model_dir = "/proj/gaia-climate/team/kirill/gaia-surrogate/lightning_logs/version_9"
model = TrainingModel.load_from_checkpoint(
            get_checkpoint_file(model_dir), map_location = "cpu",
        )
_ = model.eval()
example = torch.rand(10,164)
out = model.model(example)
traced_script_module = torch.jit.trace(model.model, example)
traced_script_module.save("traced_model.pt")
open("traced_model_arch_printed.txt","w").write(str(traced_script_module))

no stats provided, assuming will be loaded later, initializing randomly
unkown kwargs ['dataset_params']


1552

1552